# Prueba del mejor algoritmo de aprendizaje automático para la predicción.

Este notebook o libro, tarda unos 40 minutos en ejecutarse, pero ya lo hemos ejecutado y guardado los datos por usted. Sin embargo, léalo para que entienda cómo llegamos a las conclusiones que usaremos en el futuro. k takes about 40 minutes to run, but we've already run it and saved the data for you. Please read through it, though, so that you understand how we came to the conclusions we'll use moving forward.

## Seis algoritmos

Vamos a comparar seis algoritmos diferentes para determinar el mejor para producir un modelo preciso para nuestras predicciones.

### Regresión logística

La Regresión Logística (LR) es una técnica tomada del campo de la estadística. Es el método de consulta para problemas de clasificación binaria (problemas con dos valores de clase).
![](./docs/logisticfunction.png)

La regresión logística se llama así por la función utilizada en el interior del método: La función logística. La función logística es un método probablístico utilizado para determinar si el conductor será o no el ganador. La regresión logística predice probabilidades.

### Decision Tree (Árbol de decisión)

Un árbol tiene muchas analogías en la vida real, y resulta que ha influido en una amplia área del aprendizaje automático, que abarca tanto la clasificación como la regresión. En el análisis de decisiones, un árbol de decisiones se puede utilizar para representar visual y explícitamente las decisiones y la toma de decisiones.
![](./docs/decisiontree.png)

Esta metodología se conoce más comúnmente como un "árbol de decisión" a partir de datos, y el árbol de la gráfica anterior se llama árbol de clasificación porque el objetivo es clasificar a un conductor como ganador o no.
### Random Forest

Random forest es un algoritmo de aprendizaje supervisado. El "bosque" que construye es un **conjunto de árboles de decisión**, generalmente entrenados con el método " bagging ", una combinación de modelos de aprendizaje que aumenta la precisión del resultado.
Un bosque aleatorio erradica las limitaciones de un algoritmo de árbol de decisión. Reduce el sobreajuste de los conjuntos de datos y aumenta la precisión. Genera predicciones sin requerir muchas configuraciones.

![](./docs/randomforest.png)

Esta es la diferencia entre los métodos árbol de decisión y bosque aleatorio:

![](./docs/treefortheforest.jpg)

### Support Vector Machine Algorithm (SVC)

Support Vector Machines (SVMs) son un conjunto de métodos de aprendizaje supervisado utilizados para la clasificación, regresión y detección de valores atípicos.

Las ventajas de las máquinas vectoriales de soporte son:

- Eficaz en espacios de alta dimensión
- Sigue siendo eficaz en los casos en que el número de dimensiones es mayor que el número de muestras.
- Utiliza un subconjunto de puntos de entrenamiento en la función de decisión (llamados vectores de apoyo), por lo que también es eficiente en memoria.
- Versátil: se pueden especificar diferentes funciones del kernel para la función de decisión. Se proporcionan núcleos comunes, pero también es posible especificar núcleos personalizados.

El objetivo de un SVC  (Support Vector Classifier) es ajustarse a los datos que proporciona, devolviendo un hiperplano de "mejor ajuste" que divide o categoriza sus datos.

### Gaussian Naive Bayes Algorithm

Naive Bayes es un algoritmo de clasificación para problemas de clasificación binarios (dos clases) y multi-clase. La técnica es más fácil de entender cuando se describe utilizando valores de entrada binarios o categóricos. La representación utilizada para el naive Bayes son las probabilidades.

Una lista de probabilidades se almacena en un archivo para un modelo de Naive Bayes. Esto incluye:

- **Class Probabilities:** Las probabilidades de cada clase en el conjunto de datos de entrenamiento.
- **Conditional Probabilities:** Las probabilidades condicionales de cada valor de entrada dado cada valor de clase.

Naive Bayes se puede extender a atributos de valor real, más comúnmente asumiendo una distribución gaussiana. Esta extensión de Naive Bayes se llama Gaussian Naive Bayes. Se pueden usar otras funciones para estimar la distribución de los datos, pero la distribución gaussiana (o normal) es la más fácil de trabajar porque solo necesita estimar la media y la desviación estándar de sus datos de entrenamiento.

### k Nearest Neighbor Algorithm (kNN)

El k-Nearest Neighbors (KNN) es un algoritmo de aprendizaje automático simple y supervisado que se puede usar para resolver problemas de clasificación y regresión.

KNN funciona encontrando las distancias entre una consulta y todos los ejemplos en los datos, seleccionando los ejemplos de números especificados (k) más cercanos a la consulta, luego votando por la etiqueta más frecuente (en el caso de la clasificación) o promediando las etiquetas (en el caso de la regresión).

El algoritmo kNN asume la similitud entre el nuevo caso/datos y los casos disponibles, y coloca el nuevo caso en la categoría que es más similar a las categorías disponibles.

![](./docs/knn.png)

## Análisis de los datos

### Importancia de las características o variables.

Otra gran cualidad del algoritmo de los bosque aleatorios (random forest) es que es fácil medir la importancia relativa de cada característica para la predicción.

La biblioteca Python Scikit-learn proporciona una gran herramienta para esto que mide la importancia de una característica al observar cuánto reducen la impureza los nodos de árbol que usan esa característica en todos los árboles del bosque. Calcula esta puntuación automáticamente para cada característica después del entrenamiento y escala los resultados para que la suma de toda la importancia sea igual a una.

### Visualización de datos al crear un modelo

¿Cómo visualizas la influencia de los datos? ¿Cómo se enmarca el problema?

Una herramienta importante en el kit de herramientas del científico de datos es el poder de visualizar datos utilizando varias bibliotecas excelentes como Seaborn o MatPlotLib. Representar sus datos visualmente podría permitirle descubrir correlaciones ocultas que puede aprovechar. Sus visualizaciones también pueden ayudarlo a descubrir sesgos o datos desequilibrado.

![](./docs/visualization.png)

### División del conjunto de datos


Antes del entrenamiento, se debe dividir su conjunto de datos en dos o más partes de tamaño desigual que aún representen bien los datos.

1. Training. Esta parte del conjunto de datos se ajusta a su modelo para entrenarlo. Este conjunto constituye la mayoría del conjunto de datos original.
2. Testing. Un conjunto de datos de prueba es un grupo independiente de datos, a menudo un subconjunto de los datos originales, que se utiliza para confirmar el rendimiento del modelo que ha creado.
3. Validating. Un conjunto de validación es un grupo independiente más pequeño de ejemplos que se utiliza para ajustar los hiperparámetros o la arquitectura del modelo para mejorar el modelo. Dependiendo del tamaño de sus datos y de la pregunta que esté haciendo, es posible que no necesite crear este tercer conjunto.